In [3]:
import gurobipy as gp

In [9]:
model = gp.Model("P3")

Restricted license - for non-production use only - expires 2026-11-23


In [11]:
import pandas as pd

In [13]:
file_path = 'P3.XLSX'
C = pd.read_excel(file_path, sheet_name='C', index_col=0)
D = pd.read_excel(file_path, sheet_name='D', index_col=0)
P = pd.read_excel(file_path, sheet_name='P', index_col=0)
f = pd.read_excel(file_path, sheet_name='f', index_col=0)

In [15]:
C

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,50,70,65,80,55,60,75,90,85,95,50,65,70,60,80,85,75,90,95,100
2,30,45,35,60,50,55,60,70,75,80,40,55,60,50,65,70,60,80,85,90
3,60,80,75,85,65,70,85,100,95,105,65,80,85,75,95,100,90,105,110,115
4,40,60,55,70,60,65,75,85,80,95,55,70,75,65,85,90,80,95,100,110
5,70,90,80,95,75,80,90,110,100,120,80,95,100,90,110,115,100,115,120,130
6,55,65,60,75,65,70,85,95,90,100,60,75,80,70,85,95,80,90,100,110


In [17]:
D

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,150,195,115,180,140,125,165,185,110,200,155,140,135,190,110,170,175,120,145,155


In [19]:
P

,1
1,1500
2,1000
3,1300
4,1200
5,1700
6,800


In [21]:
f

,1
1,16000
2,15000
3,17500
4,13000
5,25000
6,13500


In [23]:
# Sets

Supplier = range(1, len(P)+1) #(1, 6+1) (6 suppliers)
Retailer = range(1, len(D.columns)+1) #(1, 20+1) (20 retailers)

In [25]:
# Var

x=model.addVars(Supplier, Retailer, vtype=gp.GRB.INTEGER, name="x")
y=model.addVars(Supplier, vtype=gp.GRB.BINARY, name="y")

In [29]:
# Obj

model.setObjective(gp.quicksum (  gp.quicksum ( C.loc[s,r] * x[s,r] for r in Retailer ) 
                                + f.loc[s,1] * y[s] for s in Supplier  ) , gp.GRB.MINIMIZE )

In [31]:
#Const

for r in Retailer:
    model.addConstr( gp.quicksum( x[s,r] for s in Supplier ) >= D.loc[1,r] , "C1"  )

for s in Supplier:
    model.addConstr( gp.quicksum (x[s,r] for r in Retailer ) <= P.loc[s,1] * y[s]  , "C2"  )

for s in Supplier:
    if s <= 5 :
        model.addConstr(y[s] + y[s+1] <=1 , "C3" )

In [33]:
# Solve

model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 31 rows, 126 columns and 256 nonzeros
Model fingerprint: 0xfd189e37
Variable types: 0 continuous, 126 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+01, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 271825.00000
Presolve time: 0.00s
Presolved: 31 rows, 126 columns, 256 nonzeros
Variable types: 0 continuous, 126 integer (6 binary)

Root relaxation: objective 2.560469e+05, 46 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 256046.875    0  

In [35]:
#DF
df_x = pd.DataFrame( [  [ x[s,r].x for r in Retailer ] for s in Supplier ] , index=Supplier , columns = Retailer )
df_y = pd.DataFrame( [   y[s].x for s in Supplier ] , index=Supplier , columns = ["1"] )
 
    
with pd.ExcelWriter("R.xlsx", engine="openpyxl") as writer:
    df_x.to_excel(writer, sheet_name = "XR", index=True )
    df_y.to_excel(writer, sheet_name = "YR", index=True )

In [37]:
df_x

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,-0.0,-0.0,-0.0,-0.0,140.0,125.0,-0.0,-0.0,-0.0,-0.0,155.0,140.0,135.0,190.0,110.0,170.0,35.0,-0.0,145.0,155.0
2,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
3,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
4,150.0,195.0,115.0,80.0,-0.0,-0.0,165.0,185.0,110.0,200.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
5,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
6,-0.0,-0.0,-0.0,100.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,140.0,120.0,0.0,-0.0


In [39]:
df_y

,1
1,1.0
2,-0.0
3,-0.0
4,1.0
5,-0.0
6,1.0
